In [5]:
import pandas as pd
import numpy as np
import json
import requests
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import urllib.request as req
import time
from urllib.request import urlopen
from scipy.stats import norm
import quandl


In [6]:
def get_historical_prices(symbol):
    site = 'https://api.iextrading.com/1.0/stock/' + symbol + '/chart/5y'
    data = json.loads(req.urlopen(site).read().decode())
    data = json.dumps(data)
    data = pd.read_json(data)
    data.index = pd.DatetimeIndex(data['date'])
    return data

def clean_options_df(df, spot_price, call_put_flag):
    df = df.drop(['contractSize', 'currency'], axis=1)
    df['expiration'] = pd.to_datetime(df['expiration'], unit='s')
    today = pd.to_datetime(pd.Timestamp.today().strftime('%D'))
    df['days to expiration'] = (df['expiration'] - today).dt.days
    df['B/A Spread'] = df['ask'] - df['bid']

    df = df.dropna()
    df = df.reset_index()
    df = df.drop('index', axis=1)
    df['lastTradeDate'] = pd.to_datetime(df['lastTradeDate'], unit='s')
    df['Days Since Last Trade'] = -(df['lastTradeDate'] - today).dt.days
    df['Spot Price'] = spot_price
    df['Call_Put_Flag'] = call_put_flag
    return df

In [36]:
def optionsdata(ticker):
    site = 'https://query2.finance.yahoo.com/v7/finance/options/' + ticker
    with urlopen(site) as response:
        DATA = response.read().decode()
        DATA = json.loads(DATA)

    Expirations = DATA["optionChain"]['result'][0]['expirationDates']
    Expiration_Dates = pd.to_datetime(Expirations, unit='s')
    Strikes = pd.Series(DATA["optionChain"]['result'][0]['strikes'])
    strikes = DATA["optionChain"]['result'][0]['strikes']
    ###collecting stock data####
    index = []
    entry = []
    for a in DATA["optionChain"]['result'][0]['quote'].items():
        index.append(a[0])
        entry.append(a[1])

    todays_quotes = pd.DataFrame(entry, index=index, columns=['Entry'])
    Master_List = []
    Master_Calls = []
    Master_Puts = []
    for expiration in Expirations:
        try:

            expiry = str(expiration)
            expiration_date = pd.to_datetime(expiration, unit='s')
            new_site = 'https://query2.finance.yahoo.com/v7/finance/options/' + ticker + '?date=' + expiry

            with urlopen(new_site) as response:
                DATA = response.read().decode()
                DATA = json.loads(DATA)
                Master_List.append(DATA)
                Call_df = pd.read_json(json.dumps(DATA['optionChain']['result'][0]['options'][0]['calls']))
                Put_df = pd.read_json(json.dumps(DATA['optionChain']['result'][0]['options'][0]['puts']))
                Master_Calls.append(Call_df)
                Master_Puts.append(Put_df)
        except:
            Exception

    Calls = pd.concat(Master_Calls)
    Puts = pd.concat(Master_Puts)
    spot = todays_quotes.loc['regularMarketPrice'].iloc[0]

    # Clean Call Data
    Calls = clean_options_df(Calls, spot, 'Call')
    Puts = clean_options_df(Puts, spot, 'Put')
    Calls[['delta', 'gamma', 'vega', 'theta', 'rho', 'lambdaa','epsilon','vanna','charm','volga','DvegaDtime','veta','color','ultima','speed','zomma']] = Calls.apply(greeks, axis=1)
    Puts[['delta', 'gamma', 'vega', 'theta', 'rho','lambdaa','epsilon','vanna','charm','volga','DvegaDtime','veta','color','ultima','speed','zomma']] = Puts.apply(greeks, axis=1)
    Calls[['$/Delta', '$/Gamma']] = Calls.apply(per_dollar_basis, axis=1)
    Puts[['$/Delta', '$/Gamma']] = Puts.apply(per_dollar_basis, axis=1)

    return Calls, Puts

def per_dollar_basis(df):
    delta = df['delta']
    gamma = df['gamma']
    try:
        price = df['lastPrice']
    except:
        price = df['Ask']
    try:
        dol_per_delta = price / delta
        dol_per_gamma = price / gamma
    except:
        dol_per_delta = np.nan
        dol_per_gamma = np.nan

    return pd.Series([dol_per_delta, dol_per_gamma])


def greeks(df):
    rate = 0.015
    q = 0
    spot = df['Spot Price']
    strike = df['strike']
    vol = df['impliedVolatility']
    maturity = df['days to expiration']
    Call_Put_Flag = df['Call_Put_Flag']
    


    """ask, bid, change, contractSymbol, expiration,vol, inTheMoney, lastPrice, lastTradeDate,
    openInterest, percentChange, strike, volume,maturity, BA_Spread, DaysSinceLastTrade,
    spot, Call_Put_Flag = list(column_list)"""
    d1 = (np.log(spot / strike) + (maturity / 365) * (rate - q + (vol ** 2) / 2)) / (vol * np.sqrt(maturity / 365))
    d2 = d1 - vol * np.sqrt(maturity / 365)
    gamma = ((np.exp(-q * maturity / 365) / (spot * vol * np.sqrt(maturity / 365))) * 1 / np.sqrt(2 * np.pi)) * np.exp(
        (-d1 ** 2) / 2)
    vega = (((spot * np.exp(-q * maturity / 365)) * np.sqrt(maturity / 365)) / 100) * 1 / np.sqrt(2 * 3.14159) * np.exp(
        (-d1 ** 2) / 2)
    vanna=np.exp(-q * maturity / 365)*np.sqrt(maturity/365)*(d2/vol)*np.exp(-(d1**2)/2)/(2*np.pi)
    
    volga = (spot*(np.exp(q*maturity/365))*np.sqrt(maturity/365)*(np.exp(-(d1**2)/2)*d1*d2)/(np.sqrt(2 * np.pi))*vol)
    ultima = - vega/(vol*vol) *(d1*d2*(1-d1*d2)+d1*d1+d2*d2)
    color = - np.exp(-q * maturity / 365) * 1 / np.sqrt(2 * 3.14159) * np.exp((-d1 ** 2) / 2) * 1 / (2 * spot * (maturity/365) * vol * np.sqrt(maturity / 365)) * (2* q * (maturity/365) + 1 + (2*(rate-q)*(maturity/365)-d2*vol*np.sqrt(maturity/365)/(vol*np.sqrt(maturity/365))*d1)) 
    zomma = gamma*((d1*d2-1)/vol)
    speed = - gamma / spot *(d1/(vol*np.sqrt(maturity / 365))+1) 
    veta = - spot * np.exp(-q * maturity / 365) * 1 / np.sqrt(2 * 3.14159) * np.exp((-d1 ** 2) / 2) * np.sqrt(maturity / 365) *(q + ((rate-q)*d1)/(vol*np.sqrt(maturity / 365) - (1+d1*d2)/(2*(maturity/365))))


    if Call_Put_Flag == 'Call':
        try:
            delta = np.exp(-q * maturity / 365) * norm.cdf(d1)
        except:
            delta= np.nan
        try:
            theta = ((1 / (maturity / 365)) * -((spot * vol * np.exp(-q * maturity / 365))) / 2 * np.sqrt(
            maturity / 365)) * (np.exp((-d1 ** 2) / 2)) / np.sqrt(2 * 3.14159) - rate * strike * np.exp(
            -rate * maturity / 365) * norm.cdf(d2) + q * spot * np.exp(rate * maturity / 365) * norm.cdf(d1)
        except:
            theta = np.nan
        try:
            rho = strike * maturity / 365 * np.exp(-rate * maturity / 365) * norm.cdf(d2)
        except:
            rho = np.nan
        
        try:
            epsilon = -strike*(maturity / 365)*np.exp(-q * maturity / 365)*norm.cdf(d1)
        except:
            epsilon = np.nan
        
        try: 
            part1=-q*(np.exp(-q * maturity / 365))*norm.cdf(d1)
            part2=np.exp(-q * maturity / 365)*norm.cdf(d1)*((2*(rate-q)*maturity / 365)-(d2*vol*np.sqrt(maturity/365)))/(2*(maturity/365)*vol*np.sqrt(maturity/365))
            charm=part1+part2
        except:
            charm = np.nan
       
        try: 
            DvegaDtime= 1
        except:
            DvegaDtime = np.nan
        try: 
            fair_value=spot*(np.exp(-q * maturity / 365))*norm.cdf(d1)-(np.exp(-rate * maturity / 365))*strike*norm.cdf(d2)
            lambdaa= delta*spot/fair_value
        except:
            lambdaa = np.nan
                 
      
           
    if Call_Put_Flag == 'Put':
        try:
            delta = np.exp(-rate * maturity / 365) * (norm.cdf(d1) - 1)
        except:
            delta = np.nan
        try:
            theta = ((1 / maturity / 365) * -(spot * vol * np.exp(-q * maturity / 365)) / 2 * np.sqrt(maturity / 365) * (
            np.exp((-d1 ** 2) / 2)) / np.sqrt(2 * 3.14159)) + rate * strike * np.exp(-rate * maturity / 365) * norm.cdf(
            -d2) - q * spot * np.exp(rate * maturity / 365) * norm.cdf(-d1)
        except:
            theta = np.nan
        try:
            rho = -strike * maturity / 365 * np.exp(-rate * maturity / 365) * norm.cdf(-d2)
        except:
            rho = np.nan
        
        try:
            
            epsilon = strike*(maturity / 365)*np.exp(-q * maturity / 365)*norm.cdf(-d1)
        except:
            epsilon = np.nan
        
        try: 
            part1=q*(np.exp(-q * maturity / 365))*norm.cdf(-d1)
            part2=np.exp(-q * maturity / 365)*norm.cdf(d1)*((2*(rate-q)*maturity / 365)-(d2*vol*np.sqrt(maturity/365)))/(2*(maturity/365)*vol*np.sqrt(maturity/365))
            charm=part1+part2
        except:
            charm = np.nan
        
        try: 
            DvegaDtime= 1
        except:
            DvegaDtime = np.nan
        try: 
            fair_value=(np.exp(-rate * maturity / 365))*norm.cdf(-d2)-spot*(np.exp(-q * maturity / 365))*norm.cdf(-d1)
            lambdaa= delta*spot/fair_value
        except:
            lambdaa = np.nan
    
    
    return pd.Series([delta, gamma, vega, theta, rho, lambdaa, epsilon, vanna, charm, volga, DvegaDtime,veta,color,ultima,speed,zomma])




In [47]:
calls,puts=optionsdata('TSLA')
